Initial import

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

from tensorflow import keras
import os
import re

W0426 16:08:49.338508 139735888795520 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Download actual data from REALEC

In [2]:
  dataset = tf.keras.utils.get_file(
      fname="data.tar.gz", 
      origin="http://realec.org/ajax.cgi?action=downloadCollection&collection=%2F&protocol=1", 
      extract=True)

90169344/Unknown - 28s 0us/step

In [3]:
dataset

'/root/.keras/datasets/data.tar.gz'

In [4]:
os.listdir("/root/.keras/datasets/data")

['2012-2014',
 'realec_0516.tar.gz',
 'tools.conf',
 'Academic_Writing',
 'README',
 '01_different.py',
 'errors.lst',
 'Exam_practice',
 'de',
 'old IELTS',
 'MAGOLEGO',
 'visual.conf',
 'kb_shortcuts.conf',
 'annotation.conf',
 '.json',
 'exam',
 'Vladimir Seleznev.json']

Move downloaded data to local playground

In [5]:
import shutil

shutil.copytree("/root/.keras/datasets/data","./data")

'./data'

In [0]:
os.chdir("data")

Delete german files to clean the dataset


In [0]:
shutil.rmtree("de")

Check if any texts don't have a corresponding annotation file

In [8]:
loose_files = []

for root, dirs, files in os.walk('.'):
    for file in os.listdir(root):
        if file.endswith(".txt"):
            path = root+os.sep+file
            if not os.path.isfile(path[:-4]+".ann"):
              print(path,"does not have .ann!")
              loose_files.append(path)

./Exam_practice/KP201819/ghost2.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_5/130.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_5/129.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_5/128.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_5/131.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_5/132.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_5/133.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_2/53.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_2/54.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_6/156.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_6/153.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_6/155.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_6/157.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_6/154.txt does not have .ann!
./MAGOLEGO/2019/Midterm essay/group_4/103.txt does not have .ann!
./MAGOLEGO/2019/Midter

And vice versa:

In [0]:
loose_annotations = []

for root, dirs, files in os.walk('.'):
    for file in os.listdir(root):
        if file.endswith(".ann"):
            path = root+os.sep+file
            if not os.path.isfile(path[:-4]+".txt"):
              print(path,"does not have .ann!")
              loose_annotations.append(path)

Delete the unpaired files:

In [10]:
for path in loose_files + loose_annotations:
  os.remove(path)
  print("Removed",path)

Removed ./Exam_practice/KP201819/ghost2.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_5/130.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_5/129.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_5/128.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_5/131.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_5/132.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_5/133.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_2/53.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_2/54.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_6/156.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_6/153.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_6/155.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_6/157.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_6/154.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_4/103.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_4/102.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_4/101.txt
Removed ./MAGOLEGO/2019/Midterm essay/group_1/20.txt
Removed ./MAGOLEGO/2019/Midterm essay/gro

Now let's check the collection for duplicate text files. We won't search for duplicate annotations as two hypothetically empty annotations corresponding to two different ideal texts should not be interpreted as duplicates.

In [0]:
import sys
import os
import hashlib


def chunk_reader(fobj, chunk_size=1024):
    """Generator that reads a file in chunks of bytes"""
    while True:
        chunk = fobj.read(chunk_size)
        if not chunk:
            return
        yield chunk


def get_hash(filename, first_chunk_only=False, hash=hashlib.sha1):
    hashobj = hash()
    file_object = open(filename, 'rb')

    if first_chunk_only:
        hashobj.update(file_object.read(1024))
    else:
        for chunk in chunk_reader(file_object):
            hashobj.update(chunk)
    hashed = hashobj.digest()

    file_object.close()
    return hashed


def check_for_duplicates(paths, hash=hashlib.sha1):
    duplicate_pairs = []
    
    hashes_by_size = {}
    hashes_on_1k = {}
    hashes_full = {}

    for path in paths:
        for dirpath, dirnames, filenames in os.walk(path):
            for filename in filenames:
                full_path = os.path.join(dirpath, filename)
                try:
                    # if the target is a symlink (soft one), this will 
                    # dereference it - change the value to the actual target file
                    full_path = os.path.realpath(full_path)
                    file_size = os.path.getsize(full_path)
                except (OSError,):
                    # not accessible (permissions, etc) - pass on
                    continue

                duplicate = hashes_by_size.get(file_size)

                if duplicate:
                    hashes_by_size[file_size].append(full_path)
                else:
                    hashes_by_size[file_size] = []  # create the list for this file size
                    hashes_by_size[file_size].append(full_path)

    # For all files with the same file size, get their hash on the 1st 1024 bytes
    for __, files in hashes_by_size.items():
        if len(files) < 2:
            continue    # this file size is unique, no need to spend cpy cycles on it

        for filename in files:
            try:
                small_hash = get_hash(filename, first_chunk_only=True)
            except (OSError,):
                # the file access might've changed till the exec point got here 
                continue

            duplicate = hashes_on_1k.get(small_hash)
            if duplicate:
                hashes_on_1k[small_hash].append(filename)
            else:
                hashes_on_1k[small_hash] = []          # create the list for this 1k hash
                hashes_on_1k[small_hash].append(filename)

    # For all files with the hash on the 1st 1024 bytes, get their hash on the full file - collisions will be duplicates
    for __, files in hashes_on_1k.items():
        if len(files) < 2:
            continue    # this hash of fist 1k file bytes is unique, no need to spend cpy cycles on it

        for filename in files:
            try: 
                full_hash = get_hash(filename, first_chunk_only=False)
            except (OSError,):
                # the file access might've changed till the exec point got here 
                continue

            duplicate = hashes_full.get(full_hash)
            if duplicate:
                duplicate_pairs.append((filename, duplicate))
            else:
                hashes_full[full_hash] = filename

    return duplicate_pairs

In [0]:
duplicate_pairs = check_for_duplicates(".")

In [0]:
duplicate_txts = [pair for pair in duplicate_pairs if pair[0].endswith('.txt') and pair[1].endswith('.txt')]

In [19]:
import numpy as np

dup_ex = pd.DataFrame(duplicate_txts)
dup_ex.head()

,0,1
0,/content/data/exam/exam2017_5_2/EGe_162_2.txt,/content/data/exam/exam2017/EGe_162_2.txt
1,/content/data/exam/undefined/DZu_15_1.txt,/content/data/old IELTS/IELTS2016/DZu_15_1.txt
2,/content/data/exam/exam2014/EZa_8_2.txt,/content/data/old IELTS/IELTS2015/EZa_8_2.txt
3,/content/data/exam/exam2014/MTsy_5_1.txt,/content/data/old IELTS/IELTS2015/MTsy_5_1.txt
4,/content/data/exam/exam2014/2012-2014_2/esl_00...,/content/data/2012-2014/esl_00274.txt


In [0]:
import networkx as nx
G=nx.Graph()

In [0]:
G.add_edges_from(duplicate_txts)

In [0]:
conns = list(nx.connected_components(G))

In [26]:
conn_ex = [conn for conn in conns if len(conn) > 2]
conn_ex[:10] # example of detecting more than 2 duplicates

[{'/content/data/exam/exam2014/ESha_31_1.txt',
  '/content/data/exam/exam2014/best_works/ESha_31_1.txt',
  '/content/data/old IELTS/IELTS2015/ESha_31_1.txt'},
 {'/content/data/2012-2014/esl_01163.txt',
  '/content/data/Exam_practice/AV_1_year/Book_reviews/student65_review.txt',
  '/content/data/exam/exam2014/2012-2014_5/esl_01163.txt'},
 {'/content/data/2012-2014/esl_01255.txt',
  '/content/data/Exam_practice/AV_1_year/Home_assignments/student77_final2.txt',
  '/content/data/exam/exam2014/2012-2014_6/esl_01255.txt'},
 {'/content/data/2012-2014/esl_01291.txt',
  '/content/data/2012-2014/esl_01292.txt',
  '/content/data/exam/exam2014/2012-2014_6/esl_01291.txt',
  '/content/data/exam/exam2014/2012-2014_6/esl_01292.txt'},
 {'/content/data/2012-2014/esl_00981.txt',
  '/content/data/Exam_practice/AV_1_year/Test_essays/student53_final.txt',
  '/content/data/exam/exam2014/2012-2014_4/esl_00981.txt'},
 {'/content/data/2012-2014/esl_01015.txt',
  '/content/data/Exam_practice/AV_1_year/Book_revie

Now we will clear the dataset from duplicates. We will preserve the files with the most complete annotations (judging simply by file size)

In [29]:
import operator

duplicates_removed = 0

for duplicate_group in conns:
  duplicate_group = list(duplicate_group)
  duplicate_dict = {path: os.path.getsize(path[:-4]+".ann") for path in duplicate_group}
  maxval = max(duplicate_dict.items(), key=operator.itemgetter(1))[0]
  for path in duplicate_group:
    if path != maxval:
      # print (path, maxval)
      os.remove(path)
      os.remove(path[:-4]+".ann")
      duplicates_removed += 1

print("Duplicate files removed:", duplicates_removed)

Duplicate files removed: 6482


Let's check if we have left any duplicates:

In [30]:
duplicate_pairs = check_for_duplicates(".")
duplicate_txts = [pair for pair in duplicate_pairs if pair[0].endswith('.txt') and pair[1].endswith('.txt')]
len(duplicate_txts)

0

That's it. Now we basically have a cleaned collection, free of junk files and duplicates. Here I prepare the collection for download as .tar.gz:

In [0]:
import tarfile

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))

In [0]:
import datetime
import pytz

utc_offset = lambda offset: datetime.timezone(datetime.timedelta(seconds=offset))
dt = datetime.datetime.now()
tzone = pytz.timezone('Europe/Moscow')
dt = dt.astimezone(tzone)
fname = "realec_" + dt.strftime("%d%m%y_%H%M") + ".tar.gz"

In [0]:
os.chdir("..")
make_tarfile(fname, "data")

In [34]:
os.listdir(".")

['.config', 'data', 'realec_260419_1923.tar.gz', 'sample_data']

One final step - we need to download the file from Colab's temporary workspace. Here's how it's done (uploading to your Google Drive):

In [35]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

    100% |████████████████████████████████| 993kB 19.1MB/s 


In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [37]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': fname})
uploaded.SetContentFile(fname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

W0426 16:25:35.796497 139735888795520 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNo

Uploaded file with ID 1VJufLj-heEIZ5OOpTg-Kqg4V8rnrCyuq
